<a href="https://colab.research.google.com/github/MarcoMartini75/dati/blob/master/marco_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [0]:
data = pd.read_csv("/content/dati/Womens Clothing E-Commerce Reviews.csv", index_col = 0)
# Keeping only the neccessary columns
data = data[['Title','Recommended IND']]

In [0]:
data.head()

,Title,Recommended IND
0,NaN,1
1,NaN,1
2,Some major design flaws,0
3,My favorite buy!,1
4,Flattering shirt,1


In [0]:
data.shape

(23486, 2)

In [0]:
data = data.dropna()

In [0]:
data.shape

(19676, 2)

In [0]:
data['Title'] = data['Title'].apply(lambda x: x.lower())
data['Title'] = data['Title'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Recommended IND'] == 1].size)
print(data[ data['Recommended IND'] == 0].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Title'].values)
X = tokenizer.texts_to_sequences(data['Title'].values)
X = pad_sequences(X)

32202
7150


In [0]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 11, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 11, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
Y = pd.get_dummies(data['Recommended IND']).values
#Y = data['Recommended IND']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(13182, 11) (13182, 2)
(6494, 11) (6494, 2)


In [0]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 20s - loss: 0.3363 - acc: 0.8541
Epoch 2/7
 - 18s - loss: 0.2470 - acc: 0.8936
Epoch 3/7
 - 18s - loss: 0.2186 - acc: 0.9068
Epoch 4/7
 - 18s - loss: 0.2040 - acc: 0.9130
Epoch 5/7
 - 18s - loss: 0.1947 - acc: 0.9185
Epoch 6/7
 - 18s - loss: 0.1824 - acc: 0.9229
Epoch 7/7
 - 18s - loss: 0.1762 - acc: 0.9241


In [0]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.29
acc: 0.89


In [0]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 94.82470784641069 %
neg_acc 64.23841059602648 %


In [0]:
twt = ['Some major design flaws']
#twt = ['I like a lot this pant']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=11, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0 240 991  60 456]]
negative


In [0]:
twt = ['I like a lot this pant']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=11, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0  33  54  12 341  13 193]]
positive


In [0]:
twt = ['I dont know']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=11, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0  33 167 890]]
negative
